# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [23]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [24]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API key found. {openai_api_key[:8]}")
else:
    print("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

MODEL = "gpt-4o-mini"
openai = OpenAI()



OpenAI API key found. sk-proj-


In [25]:
system_prompt = "You are the database assistant. You can help me to design a database schema. " \
    "You can also help me to write SQL queries. " \
    "You can also help me to write SQL queries in any languages such as PostgreSQL, MYSQL, Bigquery, etc. " \
    "You also know which database is very useful for specific use cases and you can give us the estimation cost. " 

In [26]:
# Let's start by making a useful function

database_prices = {"postgresql": "$100", "bigquery": "$250", "mysql": "$75"}

def get_database_price(database_name):
    print(f"Tool get_database_price called for {database_name}")
    database = database_name.lower()
    return database_prices.get(database, "Unknown")

In [27]:
get_database_price("PostgreSQL")

Tool get_database_price called for PostgreSQL


'$100'

In [28]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_database_price",
    "description": "Get the estimation price for database.  for example when a customer asks 'How much cost this database?'",
    "parameters": {
        "type": "object",
        "properties": {
            "database_name": {
                "type": "string",
                "description": "The database that the customer wants to use to",
            },
        },
        "required": ["database_name"],
        "additionalProperties": False
    }
}

In [29]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [30]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    database = arguments.get('database_name')
    price = get_database_price(database)
    response = {
        "role": "tool",
        "content": json.dumps({"database_name": database,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, database

In [31]:
def chat(message,history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response,database = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    

    return response.choices[0].message.content

In [32]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
